In [65]:
# !wget https://files.consumerfinance.gov/ccdb/complaints.csv.zip -d complaints.zip

In [66]:
# !unzip complaints.csv.zip

In [67]:
# !pip install transformers

In [68]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [69]:
df = pd.read_csv('complaints.csv', usecols = ['Product', 'Consumer complaint narrative'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2379125 entries, 0 to 2379124
Data columns (total 2 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Product                       object
 1   Consumer complaint narrative  object
dtypes: object(2)
memory usage: 36.3+ MB


In [70]:
df.head()

,Product,Consumer complaint narrative
0,"Credit reporting, credit repair services, or o...",NaN
1,Vehicle loan or lease,I contacted Ally on Friday XX/XX/XXXX after fa...
2,"Credit reporting, credit repair services, or o...",NaN
3,"Credit reporting, credit repair services, or o...",NaN
4,Checking or savings account,NaN


In [71]:
df1 = df.dropna(axis = 0)
df1.reset_index(drop=True)

df1.loc[df1['Product'] == 'Credit reporting', 'Product'] = 'Credit reporting, credit repair services, or other personal consumer reports'
df1.loc[df1['Product'] == 'Credit card', 'Product'] = 'Credit card or prepaid card'
df1.loc[df1['Product'] == 'Payday loan', 'Product'] = 'Payday loan, title loan, or personal loan'
df1.loc[df1['Product'] == 'Virtual currency', 'Product'] = 'Money transfer, virtual currency, or money service'
df1 = df1[df1.Product != 'Other financial service']

In [72]:
def print_plot(index):
    example = df1.iloc[index].values
    if len(example) > 0:
        print(example[1])
        print('Product:', example[0])

In [73]:
print_plot(42)

I presently have four student loan accounts with Navient. All four of these accounts were originally opened with Sallie Mae under the title of 'Smart Option Student Loan '. Thereafter, the accounts were assigned to Navient without notice. All four of these accounts are now incorrectly reporting missed or late payments in XX/XX/XXXX and XX/XX/XXXX ( totaling 8 missed/late payments reported ) despite my diligent efforts to have Navient correct these errors. On two occasions through XXXX and on one occasion directly with Navient by mail, I have asked Navient to correct these reporting errors that are clearly contrary to the terms of our Promissory Note. 

Pursuant to the Promissory Notes of these loans ( see attached ), repayment of these loans was to commence after the 'Interim Period ' described in the Note in Section B ( 1 ) ( a ) as " ... approximately six months after the date the student borrower graduates. '' Additionally, according to Section D ( Terms of Repayment ), Navient was 

In [74]:
import nltk
nltk.download('stopwords')

import re
from nltk.corpus import stopwords
#regular expressions for data cleaning
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '') #remove retracted info
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    text = re.sub('\d+', '', text)
    return text

df1['Consumer complaint narrative'] = df1['Consumer complaint narrative'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
print_plot(42)

presently four student loan accounts navient four accounts originally opened sallie mae title smart option student loan thereafter accounts assigned navient without notice four accounts incorrectly reporting missed late payments totaling  missed late payments reported despite diligent efforts navient correct errors two occasions one occasion directly navient mail asked navient correct reporting errors clearly contrary terms promissory note pursuant promissory notes loans see attached repayment loans commence interim period described note section b  approimately si months date student borrower graduates additionally according section terms repayment navient required send statement setting forth repayment schedule date repayment see section  date repayment schedule received electronically mail graduated new jersey accordingly school period interim period ended time separation period thereafter begun see section b  separation period lasted terms promissory note described however contrary 

In [76]:
df1.head()

,Product,Consumer complaint narrative
1,Vehicle loan or lease,contacted ally friday falling behind payments ...
5,"Credit reporting, credit repair services, or o...",thanks cfpb able get account settled lost syst...
7,"Credit reporting, credit repair services, or o...",hello complaint three credit reporting compani...
8,"Credit reporting, credit repair services, or o...",victim identity theft currently eperian accoun...
11,"Credit reporting, credit repair services, or o...",two accounts still credit history contacting c...


In [77]:
df2 = df1.sample(10000, random_state=42)

In [78]:
' '.join(df2['Consumer complaint narrative'].iloc[0].split()[0:12])

'repeatedly failed separate dad credit reporting system year said would fi breach'

In [79]:
def sample_text(x):
  if len(x.split()) > 300:
    return(' '.join(x.split()[:150]+x.split()[-150:]))
  else:
    return(x)

In [80]:
df2['Consumer complaint narrative'] = df2['Consumer complaint narrative'].apply(sample_text)

In [81]:
def check_len(x):
  return(len(x.split()))

In [82]:
len_chk = df2['Consumer complaint narrative'].apply(check_len)

In [83]:
len_chk.describe().T

count    10000.000000
mean        78.118800
std         70.812986
min          0.000000
25%         29.000000
50%         55.000000
75%        102.000000
max        300.000000
Name: Consumer complaint narrative, dtype: float64

In [84]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [85]:
tokenized = df2['Consumer complaint narrative'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [86]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [87]:
np.array(padded).shape

(10000, 454)

In [88]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10000, 454)

In [89]:
len(padded)

10000

In [90]:
padded

array([[  101,  8385,  3478, ...,     0,     0,     0],
       [  101,  6023,  1056, ...,     0,     0,     0],
       [  101,  2755,  4034, ...,     0,     0,     0],
       ...,
       [  101,  2667,  5309, ...,     0,     0,     0],
       [  101,  3531,  7901, ...,     0,     0,     0],
       [  101,  4958, 27549, ...,     0,     0,     0]])

In [ ]:
features = {}
bs = 100
for i in range(int(len(padded)/bs)):
  print(i)
  a = padded[i*bs:i*bs+bs]
  input_ids = torch.tensor(a)  
  print(len(input_ids))
  att_mask = torch.tensor(attention_mask[i*bs:i*bs+bs])

  with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=att_mask)
  features[i] = last_hidden_states[0][:,0,:].numpy()

0
100
1
100
2
100
3
100
4
100
5
100
6
100
7
100
8
100
9
100
10
100
11
100
12
100
13
100
14
100


In [ ]:
f = []
for i in range(10):
  f = f + list(features[i])

In [ ]:
f = np.array(f)

In [ ]:
f.shape

In [ ]:
# features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = pd.get_dummies(df2['Product']).values

In [ ]:
classes = df2.Product.nunique()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(f,labels, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim = 768))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
epochs = 500
batch_size = 1024

history = model.fit(X_train, Y_train, epochs=epochs,validation_split=0.1)